In [2]:
!pip install pandas

  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [1]:
import os
import json
import pandas as pd
import traceback

In [5]:
!pip install langchain_community

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 419.4 kB/s eta 0:00:05
   -------- ------------------------------- 0.5/2.3 MB 419.4 kB/s eta 0:00:05
   -------- ------------------------------- 0.5/2.3 MB 419.4 kB/s eta 0:00:05
   ------------- -------------------------- 0.8/2.3 MB 430.1 kB/s eta 0:00:04
   ------------- -------------------------- 0.8/2.3 MB 430.1 kB/s eta 0:00:04
   ----------------- ---------------------- 1.0/2.3 MB 457.5 kB/s eta 0:00:03
   ----------------- ---------------------- 1.0/2

In [2]:
from langchain.chat_models import ChatOpenAI

In [8]:
!pip install dotenv

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [67 lines of output]
      C:\Users\USER\practices\end-to-end-mcq\myenv\Lib\site-packages\setuptools\__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        Ã— python setup.py egg_info did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [1 lines of output]
            ERROR: Can not execute `setup.py` since setuptools is not available in the bui

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY")

In [6]:
from langchain_openai import ChatOpenAI

In [7]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-4o-mini", temperature=0.5)

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain
import PyPDF2

In [9]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000025F1B522330>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000025F1B5238F0>, root_client=<openai.OpenAI object at 0x0000025F1B521160>, root_async_client=<openai.AsyncOpenAI object at 0x0000025F1B522360>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [19]:
quizz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)


In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quizz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
    )

In [17]:
review_chain = LLMChain(llm=llm, prompt=quizz_evaluation_prompt, output_key='review', verbose=True)

In [20]:
generate_evaluate_chain=SequentialChain(chains=[quizz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [22]:
file_path = "C:\\Users\\USER\\practices\\end-to-end-mcq\\data.txt"


In [23]:
with open(file_path, "r") as file:
    text = file.read()

In [24]:
print(text)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [27]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [29]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke({
        "text": text,
        "number": 3,
        "subject": "Biology",
        "tone": "formal",
        "response_json": json.dumps(RESPONSE_JSON)
    })




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [30]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1661
Prompt Tokens:948
Completion Tokens:713
Total Cost:0.00057


In [31]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [33]:
quiz = json.loads(response['quiz'])

In [36]:
quiz_table_data =[]
for key, value in json.loads(response['quiz']).items():
    quiz_table_data.append([key, value['mcq'], value['correct'], value['options']['a'], value['options']['b'], value['options']['c'], value['options']['d']])

In [37]:
pd.DataFrame(quiz_table_data, columns=["Question No", "MCQ", "Option A", "Option B", "Option C", "Option D",  "Correct Answer"])

,Question No,MCQ,Option A,Option B,Option C,Option D,Correct Answer
0,1,What is the primary focus of biology as a scie...,b,The study of chemical reactions,The scientific study of life,The exploration of the universe,The analysis of physical forces
1,2,Which of the following is NOT mentioned as a m...,c,Energy processing,Evolution,Genetic engineering,Cellular organization
2,3,How do biologists approach their research acco...,b,By relying solely on theoretical models,By using the scientific method,By conducting surveys and polls,By performing only observational studies
